**type() Function**

In [26]:

DynamicClass = type('DynamicClass', (object,), {
    'attribute': 'DynamicClass',
    'method': lambda self, param1='Hello', param2='Bye' : print(f'Dynamic method with parameter: {param1} {param2} from {self.attribute}')
})

# Create an instance of the dynamically created class
instance = DynamicClass()




In [27]:
# Call the method with two argument
instance.method('Hello, World!', 'Hello again')

Dynamic method with parameter: Hello, World! Hello again from DynamicClass


In [29]:
# Call the method with no arguments
instance.method()

Dynamic method with parameter: Hello Bye from DynamicClass


**Advanced Type() Usage**

In [44]:
def create_class_with_validation(class_name, fields):
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            if key not in fields:
                raise ValueError(f"Invalid field: {key}")
            setattr(self, key, value)

    def display_fields(self):
        for field in fields:
            print(f"{field}: {getattr(self, field, None)}")

    return type(class_name, (object,), {
        '__init__': __init__,
        'fields': fields,
        'display_fields': display_fields
    })

# Example usage:
Person = create_class_with_validation('Person', ['name', 'age'])
p = Person(name="John", age=30)
p.display_fields()

name: John
age: 30


**2. Metaclass Technique**<br>
Custom Metaclass Implementation

In [45]:
class ValidationMeta(type):
    def __new__(cls, name, bases, attrs):
        ## Add custom validation logic
        attrs['validate'] = classmethod(lambda cls, data: all(
            key in data for key in cls.required_fields
        ))
        return super().__new__(cls, name, bases, attrs)

class BaseModel(metaclass=ValidationMeta):
    required_fields = []

class UserModel(BaseModel):
    required_fields = ['username', 'email']

## Validation example
print(UserModel.validate({'username': 'john', 'email': 'john@example.com'}))

True


**Database Model Generation**

In [46]:
def create_database_model(table_name, columns):
    def __init__(self, **kwargs):
        for col in columns:
            setattr(self, col, kwargs.get(col))

    return type(f'{table_name.capitalize()}Model', (object,), {
        '__init__': __init__,
        'table_name': table_name,
        'columns': columns
    })

## Dynamic database model creation
UserModel = create_database_model('users', ['id', 'username', 'email'])
product_model = create_database_model('products', ['id', 'name', 'price'])

Plugin and Extension Systems


In [47]:
class PluginManager:
    def __init__(self):
        self.plugins = {}

    def register_plugin(self, plugin_name, plugin_methods):
        plugin_class = type(f'{plugin_name.capitalize()}Plugin', (object,), plugin_methods)
        self.plugins[plugin_name] = plugin_class

    def get_plugin(self, plugin_name):
        return self.plugins.get(plugin_name)

## Plugin management example
manager = PluginManager()
manager.register_plugin('analytics', {
    'track': lambda self, event: print(f'Tracking: {event}'),
    'report': lambda self: print('Generating report')
})

analytics_plugin = manager.get_plugin('analytics')()
analytics_plugin.track('user_login')

Tracking: user_login


3. Test Case Generation

In [48]:
def generate_test_class(test_scenarios):
    class_methods = {}

    for scenario_name, test_func in test_scenarios.items():
        def create_test_method(func):
            return lambda self: func()

        class_methods[f'test_{scenario_name}'] = create_test_method(test_func)

    return type('DynamicTestCase', (object,), class_methods)

## Test scenario generation
def test_login_success():
    print("Login success scenario")

def test_login_failure():
    print("Login failure scenario")

DynamicTestCase = generate_test_class({
    'login_success': test_login_success,
    'login_failure': test_login_failure
})

test_instance = DynamicTestCase()
test_instance.test_login_success()

Login success scenario


4. API Client Generation

In [49]:
def create_api_client(base_url, endpoints):
    def generate_method(endpoint, method):
        def api_method(self, **kwargs):
            print(f"Calling {method.upper()} {base_url}{endpoint}")
            ## Actual API call implementation
        return api_method

    methods = {
        name: generate_method(endpoint['path'], endpoint['method'])
        for name, endpoint in endpoints.items()
    }

    return type('APIClient', (object,), methods)

## API client generation
github_client = create_api_client('https://api.github.com', {
    'get_user': {'path': '/users', 'method': 'get'},
    'create_repo': {'path': '/user/repos', 'method': 'post'}
})

client = github_client()
client.get_user()

Calling GET https://api.github.com/users
